<a href="https://colab.research.google.com/github/tazar09/napoleon/blob/main/napoleon_battles%5B19-nov-2023%5D.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Requirements

In [2]:
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import warnings
warnings.filterwarnings("ignore")
import datetime as dt
pd.set_option('display.max_rows', 500)
# from urllib.parse import quote, unquote
import requests
from bs4 import BeautifulSoup
from datetime import datetime as dt
import re

#Create function that converts into the right dataframe format.

In [27]:
def make_dataframe(arg_df):

  result_dic = {}
  result_dic0 = {}
  result_dic1 = {}
  result_dic2 = {}
  result_dic3 = {}
  result_dic4 = {}

  for i in range(0, len(arg_df)):

    if arg_df.iloc[i,0] == arg_df.iloc[i,1]:

      if arg_df.iloc[i,0] in ['Belligerents', 'Commanders and leaders', 'Strength', 'Casualties and losses']:
          key0 = arg_df.iloc[i,0]
          value0 = arg_df.iloc[i+1,0]
          key1 = arg_df.iloc[i,1]
          value1 = arg_df.iloc[i+1,1]
          result_dic0[key0] = value0
          result_dic1[key1] = value1

      else:
          pass

    elif arg_df.iloc[i, 0] in ['Date', 'Location', 'Result']:
      key_date = arg_df.iloc[i,0]
      value_date = arg_df.iloc[i,1]
      result_dic2[key_date] = value_date

    else:
      pass



  row0 = list(result_dic0.values())   #France
  row1 = list(result_dic1.values())   #Austria
  keyy0 = list(result_dic0.keys())    #Belligerant
  keyy1= list(result_dic1.keys())     #Belligerant

  row_date = list(result_dic2.values()) #4 december 1805
  key_date = list(result_dic2.keys())   #Date

  #1 Date Location Result
  df_date_loc_result = pd.DataFrame(columns = ['a', 'b', 'c'], data = [key_date, row_date, row_date]).T.rename(columns = {0: 'What', 1:'First team', 2:'Second team'})

  #2 Belligerants, Comamanders, Strength, Casualties
  if len(keyy0) == 4:
    df_bell_comm_str_cas = pd.DataFrame(columns = ['d', 'e','f', 'g'], data = [keyy0, row0, row1]).T.rename(columns = {0: 'What', 1:'First team', 2:'Second team'})
  elif len(keyy0) == 3:
    df_bell_comm_str_cas = pd.DataFrame(columns = ['d', 'e','f'], data = [keyy0, row0, row1]).T.rename(columns = {0: 'What', 1:'First team', 2:'Second team'})
  elif len(keyy0) == 2:
    df_bell_comm_str_cas = pd.DataFrame(columns = ['d', 'e'], data = [keyy0, row0, row1]).T.rename(columns = {0: 'What', 1:'First team', 2:'Second team'})
  else:
    df_bell_comm_str_cas = pd.DataFrame(columns = ['d'], data = [keyy0, row0, row1]).T.rename(columns = {0: 'What', 1:'First team', 2:'Second team'})

  #3 Battle
  df_battle_first = pd.DataFrame(columns = ['What','First team', 'Second team'], data = [['Battle', arg_df.columns[0], arg_df.columns[0]]])

  #4 Final concat
  df_no = pd.concat([df_battle_first, df_date_loc_result,df_bell_comm_str_cas], axis = 0)
  idx = [df_no.iloc[0,1].lower().replace(' ', '_')] * df_no.shape[0]
  df_no.index = idx

  return df_no

In [28]:
make_dataframe(pd.read_html(url)[2])

,What,First team,Second team
battle_of_millesimo,Battle,Battle of Millesimo,Battle of Millesimo
battle_of_millesimo,Date,13–14 April 1796,13–14 April 1796
battle_of_millesimo,Location,"Millesimo, present-day Italy.mw-parser-output ...","Millesimo, present-day Italy.mw-parser-output ..."
battle_of_millesimo,Result,French victory,French victory
battle_of_millesimo,Belligerents,French Republic,Kingdom of Sardinia Habsburg monarchy
battle_of_millesimo,Commanders and leaders,Napoleon Bonaparte Pierre Augereau,Giovanni di Provera Filippo del Carretto †
battle_of_millesimo,Strength,"9,000[1] 23 guns","Millesimo: 4,000[1] Cosseria: 988"
battle_of_millesimo,Casualties and losses,"Millesimo: 700[2][1] Cosseria: 1,000[3]","Millesimo: 2-3,000, 18 guns[1] Cosseria: 988, ..."


#Accessing Napoleon's list of battles.

In [29]:
url_list_of_battles = 'https://en.wikipedia.org/wiki/Military_career_of_Napoleon'
list_of_battles = pd.read_html(url_list_of_battles)[1]
list_of_battles.head(5)

,No,Date,Battle,Conflict,Opponent,Location,Outcome
0,1.0,29 Aug-19 Dec 1793,Siege of Toulon,War of the First Coalition,NaN,French Republic,Victory
1,2.0,24-28 Apr 1794,Saorgio,War of the First Coalition,NaN,Kingdom of Sardinia,Victory
2,3.0,21 September 1794,First Dego,War of the First Coalition,NaN,Kingdom of Sardinia,Victory
3,4.0,5 Oct 1795,13 Vendémiaire,French Revolution,NaN,French Republic,Victory
4,5.0,11-12 Apr 1796,Montenotte,War of the First Coalition,NaN,Kingdom of Sardinia,Victory


#Collecting hyperlinks of the battles

In [30]:
url = 'https://en.wikipedia.org/wiki/Military_career_of_Napoleon'

response = requests.get(url)
soup = BeautifulSoup(response.content, 'html.parser')

tables = soup.find_all('table')

# Iterate through the tables and look for the one with the specific class
for table in tables:
    if 'wikitable' in table.get('class', []):
        # This is the table with the class 'wikitable'
        # Now you can extract hyperlinks from this table
        hyperlinks = [a['href'] for a in table.find_all('a', href=True)]
        break

In [31]:
words_to_filter = ['Mamluk','Ottoman', 'Mediterranean', 'Sovereign','php?title','File:Flagge',
                   'Duchy', 'Sweden', 'cite', 'War', 'Kingdom', 'Spain', 'Habsburg', 'Revolution',
                   'onarch', 'Arm%C3%A9e_des_%C3%89migr%C3%A9s', 'Campaign', 'Empire', 'Russia', 'Hundred', 'Electorate']
filter_hyperlinks = [link for link in hyperlinks if not any(word in link for word in words_to_filter)]

In [32]:
print(len(filter_hyperlinks), len(hyperlinks))

80 226


In [33]:
battle_links = ['https://en.wikipedia.org/' + link for link in filter_hyperlinks]

#Process each battle URL.

In [34]:
list_of_battles_dataframes = []

for i, battle in enumerate(battle_links):

  try:
    # print(f'Processed URL: {battle}')
    tables_list = pd.read_html(battle)
    table_battle = pd.DataFrame()

    len0 = tables_list[0].shape[0]
    len1 = tables_list[1].shape[0]
    len2 = tables_list[2].shape[0]
    # len3 = tables_list[3].shape[0]

    lungimea = [len0, len1, len2]

    if max(lungimea) < 4:
      max_index = 3
    else:
      max_index = lungimea.index(max(lungimea))

    table_battle = tables_list[max_index]     #try tables[0]

    list_of_battles_dataframes.append(table_battle)
    print(i, f"Processed URL: {battle}\n")

  except Exception as e:
    print(i, f'---------> Error processing the URL: {battle}: {e}')

0 Processed URL: https://en.wikipedia.org//wiki/Siege_of_Toulon_(1793)

1 Processed URL: https://en.wikipedia.org//wiki/Second_Battle_of_Saorgio_(1794)

2 Processed URL: https://en.wikipedia.org//wiki/First_Battle_of_Dego

3 Processed URL: https://en.wikipedia.org//wiki/13_Vend%C3%A9miaire

4 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Montenotte

5 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Millesimo

6 Processed URL: https://en.wikipedia.org//wiki/Second_Battle_of_Dego

7 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Ceva

8 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Mondov%C3%AC

9 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Fombio

10 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Lodi

11 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Borghetto

12 Processed URL: https://en.wikipedia.org//wiki/Siege_of_Mantua_(1796%E2%80%9397)

13 Processed URL: https://en.wikipedia.org//wiki/Battle_of_Lonato

14 Process

#Make big dataframe

In [35]:
list_of_battles_dataframes_processed = [make_dataframe(battle) for battle in list_of_battles_dataframes[:76]]
df = pd.concat(list_of_battles_dataframes_processed, axis = 0)

df = df.reset_index(names='battle').rename(columns = {'What': 'element', 'First team': 'first', 'Second team': 'second'})

In [60]:
len(list_of_battles_dataframes)

76

We have a final list of 76 battles collected.

#Processing df

In [37]:
df['first'] = df['first'].str.replace('–', '-')
df['second'] = df['second'].str.replace('–', '-')

In [38]:
def remove_between_brackets(text):
  return re.sub(r'\[.*?\]', '', text)

def remove_between_para(text):
  return re.sub(r'\(.*?\)', '', text)

In [39]:
df['first'] = df['first'].apply(lambda x: remove_between_brackets(x))
df['second'] = df['second'].apply(lambda x: remove_between_brackets(x))

In [40]:
df['first'] = df['first'].apply(lambda x: remove_between_para(x))
df['second'] = df['second'].apply(lambda x: remove_between_para(x))

In [41]:
df['first'] = df['first'].str.replace('\xa0', ' ')
df['second'] = df['second'].str.replace('\xa0', ' ')

In [42]:
df['first'] = df['first'].str.strip()
df['second'] = df['second'].str.strip()

In [43]:
df.head(25)

,battle,element,first,second
0,siege_of_toulon,Battle,Siege of Toulon,Siege of Toulon
1,siege_of_toulon,Date,29 August - 19 December 1793,29 August - 19 December 1793
2,siege_of_toulon,Location,"Toulon, France.mw-parser-output .geo-default,....","Toulon, France.mw-parser-output .geo-default,...."
3,siege_of_toulon,Result,French Republican victory,French Republican victory
4,siege_of_toulon,Belligerents,French Republic,French Royalists French Federalists Great Bri...
5,siege_of_toulon,Commanders and leaders,Jean François Carteaux Jacques François Dugom...,Baron d'Imbert Samuel Hood Charles O'Hara ...
6,siege_of_toulon,Strength,"32,000","1,500 8,000 & 37 Ships 7,000 & 32 Ships 6,50..."
7,siege_of_toulon,Casualties and losses,"1,700 dead or wounded 9 Ships of the Line scut...","1,200 killed or wounded 700 killed or wounded..."
8,battle_of_saorgio,Battle,Battle of Saorgio,Battle of Saorgio
9,battle_of_saorgio,Date,24 to 28 April 1794,24 to 28 April 1794


#Processing Data formats

In [62]:
df1 = df.loc[(df['element'] == 'Date') & (df['first'].str.contains('-'))]
df1.head(20)

,battle,element,first,second
1,siege_of_toulon,Date,29 August - 19 December 1793,29 August - 19 December 1793
41,battle_of_millesimo,Date,13-14 April 1796,13-14 April 1796
49,second_battle_of_dego,Date,14-15 April 1796,14-15 April 1796
65,battle_of_mondovì,Date,20-22 April 1796,20-22 April 1796
73,battle_of_fombio,Date,7-9 May 1796,7-9 May 1796
97,siege_of_mantua_(1796–1797),Date,4 July - 1 August 1796 27 August 1796 - 2 Febr...,4 July - 1 August 1796 27 August 1796 - 2 Febr...
105,battle_of_lonato,Date,3-4 August 1796,3-4 August 1796
153,battle_of_arcole,Date,15-17 November 1796,15-17 November 1796
161,battle_of_rivoli,Date,14-15 January 1797,14-15 January 1797
177,battle_of_tarvis_(1797),Date,21-23 March 1797,21-23 March 1797


In [85]:
strcheck = "13-14 April 1796"
strcheck

'13-14 April 1796'

In [92]:
len(strcheck.split('-', maxsplit = 3))

2

In [134]:
make_date(strcheck)[0]

datetime.date(1796, 4, 13)

In preprocessing the Date data we need different rules for each type of data format. We will create multiple functions.

In [130]:
def make_date(strdate):

  check_count =  strdate.count('-')                 #1
  check_length = strdate.split('-', maxsplit = 3)
  length = len(strdate.split('-', maxsplit = 3))   #2
  check_first =  len(check_length[0])               #2

  # Rule no. 1 | simple data format | "13-14 April 1796"
  # the particularity of this format is that it contains only one '-'
  # and consists of only two figures before '-' character
  # and gives a list of two elements when splitted by '-'

  if ((check_count == 1) & (length == 2) & (check_first == 2)) :

    days, month, year = map(str.strip, strdate.split())
    start, end  = days.split(sep = '-')

    startdate = start + ' ' + month + ' ' + year
    enddate = end + ' ' + month + ' ' + year

    startdate = pd.to_datetime(startdate).date()
    enddate = pd.to_datetime(enddate).date()

  # Rule no. 2 | more complex data format | "29 August - 19 December 1793"
  # defined by one '-', but a higher length before '-' since there is other Mo.

  elif:






  else:
    startdate = strdate
    enddate = strdate

  return startdate, enddate






  # elif check_length1 == 2:
  #   start, end = map(str.strip, strdate.split('-'))
  #   start = start + ' ' + end[-4:]
  #   startdate = pd.to_datetime(start)
  #   startdate = startdate.strftime('%d-%m-%Y')
  #   enddate = pd.to_datetime(end)
  #   enddate = enddate.strftime('%d-%m-%Y')

  # else:
  #   startdate = strdate
  #   enddate = strdate




#testing

In [ ]:
strdate = df1.loc[97, 'first']
strdate

In [48]:
check_length = strdate.split('-', maxsplit=3)
check_length

['4 July ', ' 1 August 1796 27 August 1796 ', ' 2 February 1797']

In [49]:
check_length = strdate.split('-', maxsplit=3)
check_length = len(check_length)
check_length

3

In [50]:
def make_date(strdate):

  check_length = strdate.split('-', maxsplit = 3)
  check_length1 = len(check_length)
  # check_length2 = len(check_length[0].split())

  if len(strdate.split()) == 3:

    days, month, year = map(str.strip, strdate.split())
    start, end  = days.split(sep = '-')
    startdate = start + ' ' + month + ' ' + year
    enddate = end + ' ' + month + ' ' + year
    startdate = dt.strptime(startdate, '%d %B %Y')
    enddate = dt.strptime(enddate, '%d %B %Y')

    startdate = pd.to_datetime(startdate)
    startdate = startdate.strftime('%d-%m-%Y')

    enddate = pd.to_datetime(enddate)
    enddate = enddate.strftime('%d-%m-%Y')




  elif check_length1 == 2:
    start, end = map(str.strip, strdate.split('-'))
    start = start + ' ' + end[-4:]
    startdate = pd.to_datetime(start)
    startdate = startdate.strftime('%d-%m-%Y')
    enddate = pd.to_datetime(end)
    enddate = enddate.strftime('%d-%m-%Y')

  else:
    startdate = strdate
    enddate = strdate

  return startdate, enddate

In [51]:
df1['first'] = df1['first'].apply(lambda x: make_date(x))


In [52]:
df1['first'] = df1['first'].apply(lambda x: x[0])

In [53]:
df1.iloc[0,2][0]

'2'

In [54]:
df1

,battle,element,first,second
1,siege_of_toulon,Date,29-08-1793,29 August - 19 December 1793
41,battle_of_millesimo,Date,13-04-1796,13-14 April 1796
49,second_battle_of_dego,Date,14-04-1796,14-15 April 1796
65,battle_of_mondovì,Date,20-04-1796,20-22 April 1796
73,battle_of_fombio,Date,07-05-1796,7-9 May 1796
97,siege_of_mantua_(1796–1797),Date,4 July - 1 August 1796 27 August 1796 - 2 Febr...,4 July - 1 August 1796 27 August 1796 - 2 Febr...
105,battle_of_lonato,Date,03-08-1796,3-4 August 1796
153,battle_of_arcole,Date,15-11-1796,15-17 November 1796
161,battle_of_rivoli,Date,14-01-1797,14-15 January 1797
177,battle_of_tarvis_(1797),Date,21-03-1797,21-23 March 1797


In [55]:
for i in range(len(df1)):
  try:
    x = make_date(df1.iloc[i, 2])
    print(i,x)
  except Exception as e:
    print(f'----------->{i}, {e}')

0 ('29-08-1793', '29-08-1793')
1 ('13-04-1796', '13-04-1796')
2 ('14-04-1796', '14-04-1796')
3 ('20-04-1796', '20-04-1796')
4 ('07-05-1796', '07-05-1796')
5 ('4 July - 1 August 1796 27 August 1796 - 2 February 1797', '4 July - 1 August 1796 27 August 1796 - 2 February 1797')
6 ('03-08-1796', '03-08-1796')
7 ('15-11-1796', '15-11-1796')
8 ('14-01-1797', '14-01-1797')
9 ('21-03-1797', '21-03-1797')
10 ('10-06-1798', '10-06-1798')
11 ('01-07-1798', '01-07-1798')
12 ('21-10-1798', '21-10-1798')
13 ('08-02-1799', '08-02-1799')
14 ('03-03-1799', '03-03-1799')
15 ('20-03-1799', '20-03-1799')
16 ('14-05-1800', '14-05-1800')
17 ('16-10-1805', '16-10-1805')
18 ('07-02-1807', '07-02-1807')
19 ('21-05-1809', '21-05-1809')
20 ('05-07-1809', '05-07-1809')
21 ('26-07-1812', '26-07-1812')
22 ('16-08-1812', '16-08-1812')
23 ('.mw-parser-output .plainlist ol,.mw-parser-output .plainlist ul{line-height:inherit;list-style:none;margin:0;padding:0}.mw-parser-output .plainlist ol li,.mw-parser-output .plainl

#Checking for errors

In [56]:
errors = []
for i, abc in enumerate(list_of_battles_dataframes):
  try:
    make_dataframe(abc)
    # print(f'processed dataframe: No. {i}')
  except Exception as e:
    print(f'----Error at dataframe No. {i}')
    errors.append(i)
print(f'\n {errors}')


 []
